In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold

from libs.dataset import BirdClefDataset

In [22]:
train_file_path = '../csv/train_files.npy'
train_files = np.load(train_file_path)
ss = pd.read_csv('../../BirdCLEF2023/data/sample_submission.csv')
birds = list(ss.columns[1:])
bird_label_map = {birds[i]:i for i in range(len(birds))}


In [92]:
np.concatenate([train_files, train_files], 0)

array(['../dataset/logmel/easmog1/XC316030.npy',
       '../dataset/logmel/easmog1/XC469902.npy',
       '../dataset/logmel/easmog1/XC434328.npy', ...,
       '../dataset/logmel/comsan/XC553458.npy',
       '../dataset/logmel/comsan/XC686330.npy',
       '../dataset/logmel/comsan/XC676382.npy'], dtype='<U38')

In [23]:
oof_df = pd.DataFrame(columns  = ['filename', 'primary_label'])

oof_df['filename'] = train_files
oof_df['primary_label'] = oof_df['filename'].apply(lambda x: x.split('/')[-2])

In [24]:
oof_df['soundname'] = oof_df['filename'].map(lambda x: os.path.join(x.split('/')[-2], x.split('/')[-1])[:-4])

In [25]:
meta_df = BirdClefDataset.get_metadata()

In [26]:
oof_df = oof_df.merge(meta_df, on='soundname', how='left')

In [27]:
oof_df[birds] = 0

In [77]:
x = 1
y = 1
result_path = ''
a = ''
assert os.path.isfile(os.path.join(result_path, 'train_oof.csv')) & len(a), 'not found train_oof.csv'

AssertionError: not found train_oof.csv

In [28]:
oof_df.head(1)

,filename,primary_label,soundname,rating,secondary_labels,abethr1,abhori1,abythr1,afbfly1,afdfly1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,../dataset/logmel/easmog1/XC316030.npy,easmog1,easmog1/XC316030,2.0,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
SEED = 1
kf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

In [64]:
df = pd.DataFrame()
for fold, (train_index, val_index) in enumerate(kf.split(oof_df, y=oof_df['primary_label'])):
    train_list = list(oof_df.iloc[train_index]['filename'].values)
    valid_list = list(oof_df.iloc[val_index]['filename'].values)
    fold_df = oof_df.iloc[val_index]
    pred = np.array([[i] * len(birds) for i in range(len(valid_list))])
    fold_df[birds] = pred 
    df = pd.concat([df, fold_df])

/home/aolab/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
<ipython-input-64-df9a9c66cbac>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_df[birds] = pred
/home/aolab/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [59]:
result_path = '../result/model=bird_sed_b1-loss_fn=focal_clip_max-aug_ver=4-duration=10-model_path=exp064_sed_2021_2022'
is_done = os.path.isfile(os.path.join(result_path, 'oof_df.csv'))
print(is_done)

True


In [58]:
os.path.join(result_path, 'train_oof.csv')

'../result/model=bird_sed_b1-loss_fn=focal_clip_max-aug_ver=4-duration=10-model_path=exp064_sed_2021_2022/train_oof.csv'

In [47]:
fold_df[birds] = pred 

<ipython-input-47-8ff44a6f4125>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fold_df[birds] = pred
/home/aolab/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [48]:
fold_df

,filename,primary_label,soundname,rating,secondary_labels,abethr1,abhori1,abythr1,afbfly1,afdfly1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,../dataset/logmel/easmog1/XC316030.npy,easmog1,easmog1/XC316030,2.0,[],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,../dataset/logmel/easmog1/XC690369.npy,easmog1,easmog1/XC690369,2.0,[],1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,../dataset/logmel/easmog1/XC266004.npy,easmog1,easmog1/XC266004,4.0,[],2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
18,../dataset/logmel/litegr/XC333992.npy,litegr,litegr/XC333992,4.0,[],3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
24,../dataset/logmel/litegr/XC441059.npy,litegr,litegr/XC441059,5.0,[],4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14408,../dataset/logmel/comsan/XC385909.npy,comsan,comsan/XC385909,4.0,[],2882,2882,2882,2882,2882,...,2882,2882,2882,2882,2882,2882,2882,2882,2882,2882
14410,../dataset/logmel/comsan/XC722381.npy,comsan,comsan/XC722381,5.0,[],2883,2883,2883,2883,2883,...,2883,2883,2883,2883,2883,2883,2883,2883,2883,2883
14422,../dataset/logmel/comsan/XC667555.npy,comsan,comsan/XC667555,3.5,[],2884,2884,2884,2884,2884,...,2884,2884,2884,2884,2884,2884,2884,2884,2884,2884
14428,../dataset/logmel/comsan/XC553458.npy,comsan,comsan/XC553458,1.0,[],2885,2885,2885,2885,2885,...,2885,2885,2885,2885,2885,2885,2885,2885,2885,2885


In [17]:
for fold, (train_index, valid_index) in enumerate(kf.split(kfold_df,y=kfold_df['primary_label'])):
    print(train_index)

[    1     2     3 ... 14426 14427 14429]
[    0     3     4 ... 14428 14429 14430]
[    0     1     2 ... 14427 14428 14430]
[    0     1     2 ... 14428 14429 14430]
[    0     1     2 ... 14428 14429 14430]


/home/aolab/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [23]:
oof_df = pd.DataFrame(columns  = ['filename', 'primary_label'])

In [25]:
valid_list

['../dataset/logmel/easmog1/XC155155.npy',
 '../dataset/logmel/easmog1/XC704478.npy',
 '../dataset/logmel/litegr/XC333952.npy',
 '../dataset/logmel/litegr/XC687077.npy',
 '../dataset/logmel/litegr/XC585639.npy',
 '../dataset/logmel/litegr/XC701677.npy',
 '../dataset/logmel/litegr/XC614656.npy',
 '../dataset/logmel/litegr/XC606694.npy',
 '../dataset/logmel/litegr/XC407045.npy',
 '../dataset/logmel/litegr/XC641657.npy',
 '../dataset/logmel/litegr/XC576949.npy',
 '../dataset/logmel/litegr/XC677279.npy',
 '../dataset/logmel/litegr/XC533882.npy',
 '../dataset/logmel/litegr/XC411010.npy',
 '../dataset/logmel/litegr/XC560736.npy',
 '../dataset/logmel/litegr/XC288883.npy',
 '../dataset/logmel/litegr/XC333874.npy',
 '../dataset/logmel/litegr/XC683454.npy',
 '../dataset/logmel/litegr/XC605411.npy',
 '../dataset/logmel/litegr/XC589946.npy',
 '../dataset/logmel/litegr/XC535552.npy',
 '../dataset/logmel/litegr/XC372948.npy',
 '../dataset/logmel/litegr/XC581128.npy',
 '../dataset/logmel/litegr/XC720

In [93]:
train_oof = pd.read_csv('/media/aolab/untitiled/workspace/Kaggle/BirdCLEF2023/result/model=bird_sed_b1-loss_fn=focal_clip_max_v2-aug_ver=4-duration=10-model_path=exp064_sed_2021_2022-make_train_oof=True-cleaning=True/train_oof.csv')

In [97]:
train_oof.isna().sum().sort_values()

filename    0
pygbat1     0
quailf1     0
ratcis1     0
raybar1     0
           ..
fatrav1     0
fatwid1     0
fislov1     0
darbar1     0
yewgre1     0
Length: 269, dtype: int64

In [83]:
file = valid_list[0]

In [84]:
file

'../dataset/logmel/easmog1/XC155155.npy'

In [85]:
oof_pred = train_oof[train_oof['soundname']==file.split('/')[-2]+'/'+file.split('/')[-1][:-4]].iloc[0]

In [88]:
oof_pred 

filename            ../dataset/logmel/easmog1/XC155155.npy
primary_label                                      easmog1
soundname                                 easmog1/XC155155
rating                                                   4
secondary_labels                                        []
                                     ...                  
yertin1                                            0.52698
yesbar1                                           0.521077
yespet1                                           0.508692
yetgre1                                           0.544989
yewgre1                                           0.561025
Name: 3, Length: 269, dtype: object

In [89]:
oof_pred[oof_pred['primary_label']] 

0.5800132155418396